In [1]:
import os
import pandas as pd 
import numpy as np
import multiprocessing # 여러 개의 일꾼 (cpu)들에게 작업을 분산시키는 역할
from multiprocessing import Pool 
from functools import partial # 함수가 받는 인자들 중 몇개를 고정 시켜서 새롭게 파생된 함수를 형성하는 역할
from data_loader import data_loader_v2 # 자체적으로 만든 data loader version 2.0 ([데이콘 15회 대회] 데이터 설명 및 데이터 불러오기 영상 참조)

from sklearn.ensemble import RandomForestClassifier
import joblib # 모델을 저장하고 불러오는 역
from datetime import datetime

import lightgbm as lgb
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm_notebook

from sklearn.metrics import log_loss

from tools import eval_summary, save_feature_importance, merge_preds, report



/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
train_folder = 'data/train/'
test_folder = 'data/test/'
train_label_path = 'data/train_label.csv'

train_list = os.listdir(train_folder)
test_list = os.listdir(test_folder)
train_label = pd.read_csv(train_label_path, index_col=0)

In [3]:
num_class = len(train_label['label'].unique())

In [4]:
# 모든 csv 파일의 상태_B로 변화는 시점이 같다라고 가정
# 하지만, 개별 csv파일의 상태_B로 변화는 시점은 상이할 수 있음
def data_loader_all_v2(func, files, folder='', train_label=None, event_time=10, nrows=60):   
    func_fixed = partial(func, folder=folder, train_label=train_label, event_time=event_time, nrows=nrows)     
    if __name__ == '__main__':
        pool = Pool(processes=multiprocessing.cpu_count()-2) 
        df_list = list(pool.imap(func_fixed, files)) 
        pool.close()
        pool.join()        
    combined_df = pd.concat(df_list)    
    return combined_df

In [5]:
# event_time = 10
# nrows = 30
# train = data_loader_all_v2(data_loader_v2, train_list, folder=train_folder, train_label=train_label, 
#                            event_time=event_time, nrows=nrows)
# print(train.shape)
# joblib.dump(train, 'data/df_train_{}_{}.pkl'.format(event_time, nrows))

In [6]:
train = joblib.load('data/df_train_10_60.pkl').reset_index()

In [7]:
# event_time = 10
# nrows = 200
# test = data_loader_all_v2(data_loader_v2, test_list, folder=test_folder, train_label=None, event_time=event_time, nrows=nrows)
# print(test.shape)
# joblib.dump(train, 'data/df_test_{}_{}.pkl'.format(event_time, nrows))

In [8]:
test = joblib.load('data/df_test_10.pkl')

In [9]:
fea_cols = [c for c in train.columns if c[0] == 'V']
len(fea_cols)

5121

In [10]:
zero_cols = joblib.load('zero_cols.bin')
fea_cols = [c for c in fea_cols if c not in zero_cols]

In [11]:
# use_cols = joblib.load('use_cols.bin')
# fea_cols = use_cols[:10]

In [12]:
# fea_cols = zero_cols

In [13]:
len(fea_cols)

2730

In [14]:
train['label'].value_counts(dropna=False)

110    1200
17     1050
114    1000
118    1000
117     950
       ... 
137      50
52       50
51       50
42       50
191      50
Name: label, Length: 198, dtype: int64

In [15]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
print('model_ts', model_ts)


params = {
    'max_depth': 3,  # the maximum depth of each tree
    'eta': 0.01,  # the training step for each iteration
    'silent':1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'eval_metric': 'mlogloss',
    'num_class': 198,
    'gpu_id': 0,
    'tree_method': 'gpu_hist',
    'nthread': 16,
    'colsample_bytree':0.5,
    'colsample_bylevel':0.5,
    'colsample_bynode':0.5,
    'max_leaves': 15,
    
}
print(params)

num_round = 10000
print('num_round:', num_round)

model_ts 20200210T182940
{'max_depth': 3, 'eta': 0.01, 'silent': 1, 'objective': 'multi:softprob', 'eval_metric': 'mlogloss', 'num_class': 198, 'gpu_id': 0, 'tree_method': 'gpu_hist', 'nthread': 16, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5, 'colsample_bynode': 0.5, 'max_leaves': 15}
num_round: 10000


In [ ]:
submit_csv = []
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, random_state=81511991154 % 2**32-1, shuffle=True)

cv = 0
for train_index, valid_index in tqdm_notebook(skf.split(train.index, train['label'].values), total=n_splits, desc = 'CV'):
    
    X_train, X_test = train.loc[train_index, fea_cols], train.loc[valid_index, fea_cols] 
    y_train, y_test = train.loc[train_index,'label'], train.loc[valid_index, 'label']    
    
    print(X_train.shape, X_test.shape)
#     print(y_train.value_counts(dropna=False))
#     print(y_test.value_counts(dropna=False))

    train_set = xgb.DMatrix(X_train, label=y_train)
    val_set = xgb.DMatrix(X_test, label=y_test, )

    evals_result = {}
    
    watchlist = [(train_set, 'training'), (val_set, 'valid_1')]

    model = xgb.train(params, train_set, num_boost_round=num_round,
                    early_stopping_rounds=100, evals=watchlist, 
                      evals_result=evals_result, verbose_eval=50,
#                       xgb_model=model
                     )


    model_tag ='{}xgb_{}_{}_{}'.format(model_ts, cv, 
                                 evals_result['valid_1']['mlogloss'][model.best_iteration-1],
                                 evals_result['training']['mlogloss'][model.best_iteration-1]
                                )
    print(model_tag)

    joblib.dump(model, 'model/{}.model'.format(model_tag))
    break

(37260, 2730) (4140, 2730)


/home/aiden/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	training-mlogloss:4.79772	valid_1-mlogloss:4.80399
Multiple eval metrics have been passed: 'valid_1-mlogloss' will be used for early stopping.

Will train until valid_1-mlogloss hasn't improved in 100 rounds.
[50]	training-mlogloss:1.92909	valid_1-mlogloss:1.99853
[100]	training-mlogloss:1.28181	valid_1-mlogloss:1.36929
[150]	training-mlogloss:0.931016	valid_1-mlogloss:1.03076
[200]	training-mlogloss:0.710379	valid_1-mlogloss:0.820445
[250]	training-mlogloss:0.560895	valid_1-mlogloss:0.680677
[300]	training-mlogloss:0.454395	valid_1-mlogloss:0.584545
[350]	training-mlogloss:0.375555	valid_1-mlogloss:0.516369
[400]	training-mlogloss:0.315478	valid_1-mlogloss:0.466308
[450]	training-mlogloss:0.268386	valid_1-mlogloss:0.429398
[500]	training-mlogloss:0.230798	valid_1-mlogloss:0.401318
[550]	training-mlogloss:0.200215	valid_1-mlogloss:0.379893
[600]	training-mlogloss:0.175286	valid_1-mlogloss:0.36341
[650]	training-mlogloss:0.154333	valid_1-mlogloss:0.350305
[700]	training-mlogloss:0.1

In [ ]:
test_set = xgb.DMatrix(test[fea_cols])
pred = model.predict(test_set)

submission = pd.DataFrame(data=pred)
submission.index = test.index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()

csv_path = 'submit/{}.csv'.format(model_tag)
submit_csv.append(csv_path)
submission.to_csv(csv_path, index=True) 

print(submission.sum(axis=1))
print(submission)
cv += 1
#     break

In [ ]:
# submissions = [
# 'submit/20200201T193822_0.42165222105307115_0.03657010393259738.csv',
# 'submit/20200201T183544_0.4354487978488266_0.04354968619883053.csv',
# 'submit/20200201T173725_0.423066834354457_0.03614391993976106.csv',
# 'submit/20200201T163343_0.4287587567261741_0.042392138498467166.csv',
# 'submit/20200201T153529_0.42326652930531944_0.04098269988118284.csv',
# 'submit/20200201T143616_0.4258237823312355_0.04651153387555587.csv',
# 'submit/20200201T134047_0.43147156765580946_0.043682031170534714.csv',
# 'submit/20200201T125939_0.42958065644660504_0.040012625819045466.csv',
# 'submit/20200201T120739_0.42070899280425217_0.03477196302416032.csv',
# 'submit/20200201T110556_0.4181683365658109_0.043843902710973416.csv',
# ]


dfs = [pd.read_csv(s) for s in submit_csv]

df_submit = pd.concat(dfs)

df_submit = df_submit.groupby('id').mean()

df_submit.to_csv('submit/{}_e{}.csv'.format(model_ts, n_splits), index=True) 

In [ ]:
df_submit.sum(1)

In [ ]:
importance_type = 'split'
impt_dict = {k:v for k, v in zip(fea_cols, model.feature_importance(importance_type=importance_type))}
# sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True)
# zero_cols = []
for k, s in sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=False):
    if s == 0:
        zero_cols.append(k)
# joblib.dump(impt_dict, f'model/{file_name}_{importance_type}.pkl')

len(zero_cols)

In [ ]:
joblib.dump(zero_cols, 'zero_cols.bin')

In [ ]:
# model = joblib.load('model/20200130T221520_2.4393985000913667_0.07225009557115544.model')

In [ ]:
# pred = model.predict(test)

# submission = pd.DataFrame(data=pred)
# submission.index = test.index
# submission.index.name = 'id'
# submission = submission.sort_index()
# submission = submission.groupby('id').mean()

# submission.to_csv('submit/{}.csv'.format(model_tag), index=True) 
# model_tag

# submission.sum(axis=1)

# submission